In [7]:
import pandas as pd #DataFrame 용
import datetime
# import tkinter.messagebox as msgbox
import joblib   #모델 저장용
import json
import os

# import defGlobal
import Preprocess as pre
import keywordManage
import Learning

import mglearn

None


'd:\\'

In [25]:

print('json파일 가져오기')
#json파일 불러오기 : 이 파일과 같은 폴더에 있을 것
path ="D:\\00.Storage\\Study\\Programing\\Project\\SmartStore\\UserFile" + "\\" + 'ConstData.json'

with open(path, encoding='utf-8') as f:
    json_obj = json.load(f)

json_obj = json_obj["Learning"]
print('json파일 가져오기 종료')

#학습할 엑셀 파일 불러오기
#테스트용 : '카테고리명', 실사용: '마켓 카테고리번호'
useColumn = json_obj["learningUseColumn"] #사용할 Column명
path = json_obj["learningFileDirectory"] + '\\' + json_obj["learningFileName"]

dfExcel = pd.read_excel(path)
dfExcel = dfExcel.drop([0]) #첫번째 설명Row 삭제
df = pd.DataFrame(dfExcel, columns= useColumn) #사용하는 Column만 추출
df = df.reset_index(drop=True)

###############################################################################################
#1차가공
print('1차가공 시작')

#Output Data 가공 : 누나꺼 전용
#쿠팡의 카테고리코드번호로 나오도록 출력
#줄바꿈(char(10))으로 split -> '*'로 split -> [1]번째 Column값으로 변경
#쿠팡이 항상 첫번재 행의 쿠팡*1234로 이루어져있다고함
# if(useColumn[0] == '마켓 카테고리번호'):
#     for idx in range(0, len(df)):
#         if(type(df.loc[idx, useColumn[0]]) == str):
#             if('쿠팡*' in df.loc[idx, useColumn[0]]):
#                 df.loc[idx, useColumn[0]] = df.loc[idx, useColumn[0]].splitlines(keepends=False)[0].split('*')[1]
#         else:
#             df.drop([idx], inplace=True)
            
#     df = df.reset_index(drop=True)

# category_dic = {}
#     for j in range(len(category_list)):
#         category_list[j]= category_list[j].split("*")
#         if category_list[j] == ['']:
#             pass
#         else:
#             category_dic[category_list[j][0]] = category_list[j][1]

#Input용 키워드 리스트 추출 및 저장
keywords = []
for row in df.loc[:, useColumn[1]]:
    keywordManage.keywordExtract(insList=keywords, strValue=row, outStr=json_obj["ExceptionWord"])

keywords = keywordManage.keywordVectorizer(keywords=keywords)
keywordManage.saveKeyword(keywords)

print('1차가공 종료')
###############################################################################################
#2차가공
print('2차가공 시작')
#전처리
#Input용 키워드 리스트 호출
dfInput = keywordManage.loadKeywordList()
dfOutput = df.loc[:, useColumn[0]]

#Input DataFrame 제작
for row in df.loc[:, useColumn[1]]:    #상품명 Column 데이터만 추출[aa bb cc, dd,ee]
    keywords = []
    keywordManage.keywordExtract(insList=keywords, strValue=row)
    dfInput = pre.createInputDf(dfInput, keywords)

#Input 데이터 증가
minCnt = json_obj["MinLearningDataCnt"]

if(dfInput.shape[0] != 0 and dfOutput.shape[0] != 0):
    while(dfInput.shape[0] < minCnt and dfOutput.shape[0] < minCnt):
        # pandas 2.0.0 이상
        dfInput = pd.concat([dfInput, dfInput])
        dfOutput = pd.concat([dfOutput, dfOutput])

        # #pandas 2.0.0 ver 미만
        # dfInput = dfInput.append(dfInput)
        # dfOutput = dfOutput.append(dfOutput)

print('2차가공 종료')

json파일 가져오기
json파일 가져오기 종료
1차가공 시작
1차가공 종료
2차가공 시작
2차가공 종료


In [26]:
x, y = mglearn.datasets.make_forge()

In [29]:
#최근 학습한모델 가져오기
model = joblib.load(Learning.getRecentModel())

clf = model.fit(['쉐보레', '트레버스', '콘솔트레이', '콘솔박스', '팔걸이', '수납', '정리함']
    , ['''쿠팡*78154
옥션*75110400
G마켓*300018233
11번가*1009666
인터파크*001820238007
스마트스토어*50003663
위메프2.0*4100348
ESM2.0*00370001001100040000*75110400*300018233
롯데ON*BC52050500
톡스토어*104105137110'''])

mglearn.plots.plot_2d_separator(clf, fill=True, eps=0.5, alpha=4)
mglearn.discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
ax.set_title("{} neighbor(s)".format(n_neighbors))
ax.set_xlabel("feature0")
ax.set_ylabel("feature1")

ValueError: Expected 2D array, got scalar array instead:
array=쉐보레 트레버스 콘솔트레이 콘솔박스 팔걸이 수납 정리함.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.